In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import torch
import os
from timeit import default_timer as timer

from sklearn import manifold
from sklearn.manifold.t_sne import trustworthiness
from sklearn.model_selection import KFold
from sklearn import neighbors
from sklearn.metrics import accuracy_score

In [2]:
from core.SimpleConvNet import SimpleConvNet
from core.GraphConvNet2 import GraphConvNet2
from core.DataEmbeddingGraph import DataEmbeddingGraph
from util.plot_embedding import plot_embedding, plot_embedding_subplot
from util.mnist_data_loader import get_train_set, get_test_set


In [3]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'gpu'
else:
    print('cuda not available')
    device = 'cpu'

cuda available


In [16]:
task_parameters = {}
task_parameters['reduction_method'] = 'spectral'
task_parameters['n_components'] = 2

net_parameters = {}
net_parameters['n_components'] = task_parameters['n_components']
net_parameters['D'] = 784 # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 10 # number of hidden layers
net_parameters['n_channels'] = 1
net_parameters['n_units_1'] = net_parameters['n_units_2'] = 50

In [5]:
filename = 'data/set_100_mnist_spectral_size_200_500.pkl'
with open(filename, 'rb') as f:
    [all_test_data] = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/set_100_mnist_spectral_size_200_500.pkl'

In [5]:
# Loss functions
def l2_norm(v):
    return np.sqrt(np.sum(np.square(v), axis=1, dtype=np.float64))

def pairwise_loss_function_1(y_true, y_pred, W):
    pairwise_loss_1 = mean_squared_error(y_true[W.row,:], y_true[W.col,:])
    pairwise_loss_2 = mean_squared_error(y_pred[W.row,:], y_pred[W.col,:])
    return np.square(pairwise_loss_1 - pairwise_loss_2)

def pairwise_loss_function_2(y_true, y_pred, W):
    pairwise_loss_1 = l2_norm(y_true[W.row,:] - y_true[W.col,:])
    pairwise_loss_2 = l2_norm(y_pred[W.row,:] - y_pred[W.col,:])
    return np.average(np.square(pairwise_loss_1 - pairwise_loss_2))

In [87]:
def one_nearest_neighbours_generalisation_error(X, y):
    kf = KFold(n_splits=10)
    kf.get_n_splits(X)
    errors = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = neighbors.KNeighborsClassifier(n_neighbors=1)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        errors.append(1- accuracy_score(y_test, y_pred))
    return np.average(errors)

In [110]:
def evaluate_net(net, all_test_data):
    n_test = len(all_test_data)
    similarity_tracker = np.zeros((n_test, ))
    trust_tracker = np.zeros((n_test, ))
    one_nn_tracker = np.zeros((n_test, ))
    time_tracker = np.zeros((n_test, ))
    
    for i in range(n_test):
        G = all_test_data[i]
        time_start = timer()
        y_pred = net.forward(G).detach().numpy()
        time_end = timer()
        time_tracker[i] = time_end - time_start
        
        similarity_tracker[i] =  pairwise_loss_function_2(G.target, y_pred, G.adj_matrix)
        X = G.data.view(G.data.shape[0], -1).numpy()
        trust_tracker[i] = trustworthiness(X, y_pred, n_neighbors=5)
        one_nn_tracker[i] = one_nearest_neighbours_generalisation_error(y_pred, G.labels.numpy())
    return similarity_tracker, trust_tracker, one_nn_tracker, time_tracker

In [111]:
def evaluate_original_embedding(all_test_data):
    n_test = len(all_test_data)
    trust_tracker = np.zeros((n_test, ))
    one_nn_tracker = np.zeros((n_test, ))
    time_tracker = np.zeros((n_test, )) 
    
    for i in range(n_test):
        G = all_test_data[i]
        X = G.data.view(G.data.shape[0], -1).numpy()
        time_start = timer()
        embedder = manifold.SpectralEmbedding(n_components=2, random_state=0,
                                              eigen_solver="arpack")
        X_spectral = embedder.fit_transform(X)
        time_end = timer()
        time_tracker[i] = time_end - time_start
        
        trust_tracker[i] = trustworthiness(X, X_spectral, n_neighbors=5)
        one_nn_tracker[i] = one_nearest_neighbours_generalisation_error(X_spectral, G.labels.numpy())
    return trust_tracker, one_nn_tracker, time_tracker

In [112]:
def evaluate_tsne_embedding(all_test_data):
    n_test = len(all_test_data)
    trust_tracker = np.zeros((n_test, ))
    one_nn_tracker = np.zeros((n_test, ))
    time_tracker = np.zeros((n_test, ))
    
    for i in range(n_test):
        G = all_test_data[i]
        X = G.data.view(G.data.shape[0], -1).numpy()
        time_start = timer()
        embedder = manifold.TSNE(n_components=2, init='pca', random_state=0)
        X_tsne = embedder.fit_transform(X)
        time_end = timer()
        time_tracker[i] = time_end - time_start
        
        trust_tracker[i] = trustworthiness(X, X_tsne, n_neighbors=5)
        one_nn_tracker[i] = one_nearest_neighbours_generalisation_error(X_tsne, G.labels.numpy())
    return trust_tracker, one_nn_tracker, time_tracker

In [113]:
# Spectral embeddings
trust_tracker, one_nn_tracker, time_tracker = evaluate_original_embedding(all_test_data)
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Trustworthy score = 0.7582520548568904
1-NN generalisation error = 0.5524332002099447
Average time to compute (s) = 0.17640546113987512


In [116]:
# t-SNE embeddings
trust_tracker, one_nn_tracker, time_tracker = evaluate_tsne_embedding(all_test_data)
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Trustworthy score = 0.955044333450673
1-NN generalisation error = 0.18022943449431011
Average time to compute (s) = 5.86404038087021


In [114]:
# Simple conv net
net = SimpleConvNet(net_parameters)
root = 'results/mnist_spectral1/'
filename = root + 'conv_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

similarity_tracker, trust_tracker, one_nn_tracker, time_tracker = evaluate_net(net, all_test_data)
print("Similarity loss = {}".format(np.average(similarity_tracker)))
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))

Similarity loss = 0.0709895799395694
Trustworthy score = 0.5501872305425336
1-NN generalisation error = 0.8476193517441856
Average time to compute (s) = 0.7939383930801704


In [115]:
# Graph net
net = GraphConvNet2(net_parameters)
root = 'results/mnist_spectral1/'
filename = root + 'graph_net5.pkl'
checkpoint = torch.load(filename, map_location=device)
net.load_state_dict(checkpoint['state_dict'])

similarity_tracker, trust_tracker, one_nn_tracker, time_tracker = evaluate_net(net, all_test_data)
print("Similarity loss = {}".format(np.average(similarity_tracker)))
print("Trustworthy score = {}".format(np.average(trust_tracker)))
print("1-NN generalisation error = {}".format(np.average(one_nn_tracker)))
print("Average time to compute (s) = {}".format(np.average(time_tracker)))


nb of hidden layers= 10
dim of layers (w/ embed dim)= [784, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]


Similarity loss = 0.02278103170456813
Trustworthy score = 0.7234139254958396
1-NN generalisation error = 0.6269990045456791
Average time to compute (s) = 0.3408213357401837


In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [17]:
# Simple conv net
net = SimpleConvNet(net_parameters)   
count_parameters(net)

68952

In [13]:
# Graph net
net = GraphConvNet2(net_parameters)   
count_parameters(net)

298102

In [14]:
net = SimpleConvNet(net_parameters)   
count_parameters(net)

110658

In [15]:
net.nb_param

<bound method SimpleConvNet.nb_param of SimpleConvNet(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=3136, out_features=2, bias=True)
)>